In [1]:
#Install Dependencies
!pip install tensorflow==2.15.0 opencv-python mediapipe scikit-learn matplotlib tflite==2.10.0

In [1]:
#Import Dependencies
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [3]:
#Pose detection and Image color conversion function (BGR to RGB)
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image) #Mediapipe model prediction based on processed image
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)) #Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw right hand connections

In [6]:
#Start Video Capture, change number to change webcam used
cap = cv2.VideoCapture(0)

#Access holistic model using with statement
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Reading and Sending Feed
    while cap.isOpened():
        ret, frame = cap.read()

        #Setting up detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        #Drawing landmarks on feed
        draw_landmarks(image, results)
        
        cv2.imshow('OpenCV Feed', image)

        #setting key to stop live feed
        if cv2.waitKey(10) & 0xFF == ord('x'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [5]:
#Keypoint Extraction of frame
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z,] for res in results.face_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(1404)
    left_arm = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_arm = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, left_arm, right_arm])

In [6]:
#Path for Data
DATA_PATH = os.path.join("MP_Data")
actions = np.array(['hello', 'thanks', 'what', 'is', 'your', 'name'])

#number of sequences
no_sequences = 30
#frames of extracted data
sequence_length = 30

In [9]:
#Making folders for each action and data point
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [10]:
#Start Video Capture, change number to change webcam used
cap = cv2.VideoCapture(0)

#Access holistic model using with statement
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
        
                ret, frame = cap.read()

                #Setting up detection
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                #Drawing landmarks on feed
                draw_landmarks(image, results)
                
                #collection and waiting logic for sequences
                if frame_num == 0:
                    cv2.putText(image, "STARTING COLLECTION", (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, "Collecting frames for {} Video no. {}".format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, "Collecting frames for {} Video no. {}".format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                #Export Keypoints into files
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                
                cv2.imshow('OpenCV Feed', image)

                #setting key to stop live feed
                if cv2.waitKey(10) & 0xFF == ord('x'):
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
#Creating a label for each action
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [10]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [13]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


6/6 [==============================] - 3s 41ms/step - loss: 3.3105 - categorical_accuracy: 0.1170
Epoch 2/2000
6/6 [==============================] - 0s 41ms/step - loss: 1.8261 - categorical_accuracy: 0.1696
Epoch 3/2000
6/6 [==============================] - 0s 41ms/step - loss: 2.1943 - categorical_accuracy: 0.1520
Epoch 4/2000
6/6 [==============================] - 0s 42ms/step - loss: 2.3292 - categorical_accuracy: 0.1170
Epoch 5/2000
6/6 [==============================] - 0s 41ms/step - loss: 1.7996 - categorical_accuracy: 0.1988
Epoch 6/2000
6/6 [==============================] - 0s 41ms/step - loss: 1.8046 - categorical_accuracy: 0.1170
Epoch 7/2000
6/6 [==============================] - 0s 44ms/step - loss: 1.7913 - categorical_accuracy: 0.1170
Epoch 8/2000
6/6 [==============================] - 0s 40ms/step - loss: 1.7755 - categorical_accuracy: 0.2749
Epoch 9/2000
6/6 [==============================] - 0s 38ms/step - loss: 1.7598 - categorical_accuracy: 0.2865

6/6 [==============================] - 0s 41ms/step - loss: 0.4506 - categorical_accuracy: 0.8129
Epoch 143/2000
6/6 [==============================] - 0s 41ms/step - loss: 0.3714 - categorical_accuracy: 0.8596
Epoch 144/2000
6/6 [==============================] - 0s 41ms/step - loss: 0.3040 - categorical_accuracy: 0.8947
Epoch 145/2000
6/6 [==============================] - 0s 41ms/step - loss: 0.2811 - categorical_accuracy: 0.9006
Epoch 146/2000
6/6 [==============================] - 0s 41ms/step - loss: 0.2974 - categorical_accuracy: 0.8830
Epoch 147/2000
6/6 [==============================] - 0s 41ms/step - loss: 0.2522 - categorical_accuracy: 0.9006
Epoch 148/2000
6/6 [==============================] - 0s 41ms/step - loss: 0.2387 - categorical_accuracy: 0.9006
Epoch 149/2000
6/6 [==============================] - 0s 48ms/step - loss: 0.2302 - categorical_accuracy: 0.9123
Epoch 150/2000
6/6 [==============================] - 0s 41ms/step - loss: 0.2553 - categorical_accuracy: 0.900

6/6 [==============================] - 0s 49ms/step - loss: 0.9594 - categorical_accuracy: 0.6140
Epoch 287/2000
6/6 [==============================] - 0s 54ms/step - loss: 0.8467 - categorical_accuracy: 0.7135
Epoch 288/2000
6/6 [==============================] - 0s 53ms/step - loss: 0.8175 - categorical_accuracy: 0.7661
Epoch 289/2000
6/6 [==============================] - 0s 51ms/step - loss: 0.7429 - categorical_accuracy: 0.7544
Epoch 290/2000
6/6 [==============================] - 0s 57ms/step - loss: 0.6638 - categorical_accuracy: 0.7953
Epoch 291/2000
6/6 [==============================] - 0s 54ms/step - loss: 0.5842 - categorical_accuracy: 0.8246
Epoch 292/2000
6/6 [==============================] - 0s 60ms/step - loss: 0.5091 - categorical_accuracy: 0.8480
Epoch 293/2000
6/6 [==============================] - 0s 54ms/step - loss: 0.4123 - categorical_accuracy: 0.8889
Epoch 294/2000
6/6 [==============================] - 0s 54ms/step - loss: 0.4048 - categorical_accuracy: 0.853

6/6 [==============================] - 0s 67ms/step - loss: 0.0657 - categorical_accuracy: 0.9766
Epoch 431/2000
6/6 [==============================] - 0s 70ms/step - loss: 0.0588 - categorical_accuracy: 0.9825
Epoch 432/2000
6/6 [==============================] - 0s 57ms/step - loss: 0.0583 - categorical_accuracy: 0.9825
Epoch 433/2000
6/6 [==============================] - 0s 48ms/step - loss: 0.0590 - categorical_accuracy: 0.9825
Epoch 434/2000
6/6 [==============================] - 0s 61ms/step - loss: 0.0556 - categorical_accuracy: 0.9883
Epoch 435/2000
6/6 [==============================] - 0s 79ms/step - loss: 0.0620 - categorical_accuracy: 0.9825
Epoch 436/2000
6/6 [==============================] - 0s 67ms/step - loss: 0.0909 - categorical_accuracy: 0.9649
Epoch 437/2000
6/6 [==============================] - 0s 76ms/step - loss: 0.0883 - categorical_accuracy: 0.9825
Epoch 438/2000
6/6 [==============================] - 0s 69ms/step - loss: 0.0737 - categorical_accuracy: 0.970

6/6 [==============================] - 0s 71ms/step - loss: 0.1665 - categorical_accuracy: 0.9649
Epoch 575/2000
6/6 [==============================] - 0s 73ms/step - loss: 0.1510 - categorical_accuracy: 0.9708
Epoch 576/2000
6/6 [==============================] - 0s 71ms/step - loss: 0.1414 - categorical_accuracy: 0.9649
Epoch 577/2000
6/6 [==============================] - 0s 72ms/step - loss: 0.1307 - categorical_accuracy: 0.9708
Epoch 578/2000
6/6 [==============================] - 0s 72ms/step - loss: 0.1181 - categorical_accuracy: 0.9708
Epoch 579/2000
6/6 [==============================] - 0s 74ms/step - loss: 0.1178 - categorical_accuracy: 0.9649
Epoch 580/2000
6/6 [==============================] - 0s 65ms/step - loss: 0.1169 - categorical_accuracy: 0.9649
Epoch 581/2000
6/6 [==============================] - 0s 65ms/step - loss: 0.1102 - categorical_accuracy: 0.9649
Epoch 582/2000
6/6 [==============================] - 0s 63ms/step - loss: 0.1000 - categorical_accuracy: 0.964

...

6/6 [==============================] - 0s 62ms/step - loss: 1.2088e-06 - categorical_accuracy: 1.0000
Epoch 1413/2000
6/6 [==============================] - 0s 60ms/step - loss: 1.2032e-06 - categorical_accuracy: 1.0000
Epoch 1414/2000
6/6 [==============================] - 0s 55ms/step - loss: 1.1998e-06 - categorical_accuracy: 1.0000
Epoch 1415/2000
6/6 [==============================] - 0s 63ms/step - loss: 1.1949e-06 - categorical_accuracy: 1.0000
Epoch 1416/2000
6/6 [==============================] - 0s 63ms/step - loss: 1.1879e-06 - categorical_accuracy: 1.0000
Epoch 1417/2000
6/6 [==============================] - 0s 60ms/step - loss: 1.1851e-06 - categorical_accuracy: 1.0000
Epoch 1418/2000
6/6 [==============================] - 0s 68ms/step - loss: 1.1788e-06 - categorical_accuracy: 1.0000
Epoch 1419/2000
6/6 [==============================] - 0s 73ms/step - loss: 1.1719e-06 - categorical_accuracy: 1.0000
Epoch 1420/2000
6/6 [==============================] - 0s 53ms/step - lo

6/6 [==============================] - 0s 44ms/step - loss: 6.8179e-07 - categorical_accuracy: 1.0000
Epoch 1551/2000
6/6 [==============================] - 0s 45ms/step - loss: 6.7761e-07 - categorical_accuracy: 1.0000
Epoch 1552/2000
6/6 [==============================] - 0s 51ms/step - loss: 6.7691e-07 - categorical_accuracy: 1.0000
Epoch 1553/2000
6/6 [==============================] - 0s 54ms/step - loss: 6.7412e-07 - categorical_accuracy: 1.0000
Epoch 1554/2000
6/6 [==============================] - 0s 63ms/step - loss: 6.7064e-07 - categorical_accuracy: 1.0000
Epoch 1555/2000
6/6 [==============================] - 0s 61ms/step - loss: 6.6785e-07 - categorical_accuracy: 1.0000
Epoch 1556/2000
6/6 [==============================] - 0s 63ms/step - loss: 6.6715e-07 - categorical_accuracy: 1.0000
Epoch 1557/2000
6/6 [==============================] - 0s 64ms/step - loss: 6.6436e-07 - categorical_accuracy: 1.0000
Epoch 1558/2000
6/6 [==============================] - 0s 58ms/step - lo

6/6 [==============================] - 0s 67ms/step - loss: 3.9946e-07 - categorical_accuracy: 1.0000
Epoch 1689/2000
6/6 [==============================] - 0s 60ms/step - loss: 3.9946e-07 - categorical_accuracy: 1.0000
Epoch 1690/2000
6/6 [==============================] - 0s 58ms/step - loss: 3.9667e-07 - categorical_accuracy: 1.0000
Epoch 1691/2000
6/6 [==============================] - 0s 54ms/step - loss: 3.9527e-07 - categorical_accuracy: 1.0000
Epoch 1692/2000
6/6 [==============================] - 0s 54ms/step - loss: 3.9388e-07 - categorical_accuracy: 1.0000
Epoch 1693/2000
6/6 [==============================] - 0s 59ms/step - loss: 3.8970e-07 - categorical_accuracy: 1.0000
Epoch 1694/2000
6/6 [==============================] - 0s 60ms/step - loss: 3.8760e-07 - categorical_accuracy: 1.0000
Epoch 1695/2000
6/6 [==============================] - 0s 60ms/step - loss: 3.8760e-07 - categorical_accuracy: 1.0000
Epoch 1696/2000
6/6 [==============================] - 0s 65ms/step - lo

6/6 [==============================] - 0s 67ms/step - loss: 2.3493e-07 - categorical_accuracy: 1.0000
Epoch 1827/2000
6/6 [==============================] - 0s 65ms/step - loss: 2.3354e-07 - categorical_accuracy: 1.0000
Epoch 1828/2000
6/6 [==============================] - 0s 48ms/step - loss: 2.3214e-07 - categorical_accuracy: 1.0000
Epoch 1829/2000
6/6 [==============================] - 0s 44ms/step - loss: 2.3214e-07 - categorical_accuracy: 1.0000
Epoch 1830/2000
6/6 [==============================] - 0s 56ms/step - loss: 2.3145e-07 - categorical_accuracy: 1.0000
Epoch 1831/2000
6/6 [==============================] - 0s 51ms/step - loss: 2.3075e-07 - categorical_accuracy: 1.0000
Epoch 1832/2000
6/6 [==============================] - 0s 57ms/step - loss: 2.3075e-07 - categorical_accuracy: 1.0000
Epoch 1833/2000
6/6 [==============================] - 0s 45ms/step - loss: 2.3005e-07 - categorical_accuracy: 1.0000
Epoch 1834/2000
6/6 [==============================] - 0s 57ms/step - lo

6/6 [==============================] - 0s 64ms/step - loss: 1.4361e-07 - categorical_accuracy: 1.0000
Epoch 1965/2000
6/6 [==============================] - 0s 60ms/step - loss: 1.4361e-07 - categorical_accuracy: 1.0000
Epoch 1966/2000
6/6 [==============================] - 0s 54ms/step - loss: 1.4291e-07 - categorical_accuracy: 1.0000
Epoch 1967/2000
6/6 [==============================] - 0s 61ms/step - loss: 1.4082e-07 - categorical_accuracy: 1.0000
Epoch 1968/2000
6/6 [==============================] - 0s 63ms/step - loss: 1.4082e-07 - categorical_accuracy: 1.0000
Epoch 1969/2000
6/6 [==============================] - 0s 63ms/step - loss: 1.4082e-07 - categorical_accuracy: 1.0000
Epoch 1970/2000
6/6 [==============================] - 0s 51ms/step - loss: 1.4082e-07 - categorical_accuracy: 1.0000
Epoch 1971/2000
6/6 [==============================] - 0s 41ms/step - loss: 1.4082e-07 - categorical_accuracy: 1.0000
Epoch 1972/2000
6/6 [==============================] - 0s 45ms/step - lo

In [15]:
model.load_weights('ASL_model.h5')

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596774 (2.28 MB)
Trainable params: 596774 

In [37]:
print(os.getcwd())

C:\Users\shrim\OneDrive\Desktop\Design Product


In [29]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('image_model.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\shrim\AppData\Local\Temp\tmpxff9blu2\assets


INFO:tensorflow:Assets written to: C:\Users\shrim\AppData\Local\Temp\tmpxff9blu2\assets


627536

In [17]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [18]:
yhat = model.predict(x_test)

1/1 [==============================] - 1s 619ms/step


In [19]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [20]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[8, 0],
        [0, 1]],

       [[7, 0],
        [0, 2]],

       [[6, 0],
        [0, 3]],

       [[8, 0],
        [0, 1]],

       [[8, 0],
        [0, 1]],

       [[8, 0],
        [0, 1]]], dtype=int64)

In [21]:
accuracy_score(ytrue, yhat)

1.0

Real Time Testing

In [23]:
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)

#Access holistic model using with statement
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Reading and Sending Feed
    while cap.isOpened():
        ret, frame = cap.read()

        #Setting up detection
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        #Drawing landmarks on feed
        draw_landmarks(image, results)
        
        #prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence)==30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
        #Rendering logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
                    
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        
        cv2.imshow('SignCam', image)

        #setting key to stop live feed
        if cv2.waitKey(10) & 0xFF == ord('x'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 63ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
thanks
<class 'media